<a href="https://colab.research.google.com/github/rajdeepbanerjee-git/JNCLectures_Intro_to_ML/blob/main/Week12/2025/Lec12_adaboost_m1_cleaned_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdaBoost.M1 from Scratch
This notebook demonstrates a simple implementation of the AdaBoost.M1 algorithm using decision stumps as weak learners.

We'll use the **Iris dataset** (binary classification: *versicolor* vs *virginica*) and evaluate how boosting improves model performance.

### References:
- Dataset: [UCI Iris Dataset on Kaggle](https://www.kaggle.com/datasets/uciml/iris)
- Algorithm: *The Elements of Statistical Learning* by Hastie, Tibshirani, Friedman


## AdaBoost.M1 Algorithm Steps
1. **Initialize** all observation weights equally.
2. For each round *m = 1 to M*:
   - Sample training data using current weights.
   - Train a weak learner (e.g., decision stump).
   - Compute the error rate and corresponding alpha value.
   - Update observation weights: increase for misclassified samples.
   - Normalize weights.
3. Final model aggregates the weighted predictions.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score


## Load and Preprocess Data

In [3]:
from sklearn.datasets import load_iris
import pandas as pd

# Load iris dataset
iris = load_iris(as_frame=True)
df = iris.frame
df.rename(columns={'target': 'Label'}, inplace=True)
df['Label'] = df['Label'].map(dict(enumerate(iris.target_names)))

# Filter only versicolor and virginica
df = df[df['Label'].isin(['versicolor', 'virginica'])].copy()

# Relabel as +1 and -1
df['Label'] = df['Label'].map({'versicolor': 1, 'virginica': -1})

X = df.drop('Label', axis=1)
y = df['Label']

print(X.shape, y.shape)

(100, 4) (100,)


## Train a Weak Learner (Base Model)

In [4]:
base_tree = DecisionTreeClassifier(max_depth=1, random_state=0)
base_tree.fit(X, y)
y_pred = base_tree.predict(X)
f1_base = f1_score(y, y_pred)
print(f"F1 Score (Base): {f1_base:.4f}")

F1 Score (Base): 0.9423


## Define Boosting Functions

In [9]:
rng = np.random.RandomState(42)

def get_bootstrap_data(X_train, y_train, weights):
    indices = X_train.index
    sampled_indices = rng.choice(indices, size=len(indices), replace=True, p=weights)
    return X_train.loc[sampled_indices], y_train.loc[sampled_indices], sampled_indices

def get_updated_weights(y_true, y_pred, weights):
    misclassified = (y_true != y_pred)
    err = np.dot(weights, misclassified)
    alpha = np.log((1 - err) / err)
    weights = weights * np.exp(alpha * misclassified)
    return weights / weights.sum(), alpha

## Run AdaBoost.M1 for M Iterations

In [12]:
M = 10  # number of boosting rounds
n = len(X)
weights = np.ones(n) / n
alpha_list = []
y_pred_list = []

for m in range(M):
    X_bs, y_bs, _ = get_bootstrap_data(X, y, weights)
    clf = DecisionTreeClassifier(max_depth=1, random_state=0)
    clf.fit(X_bs, y_bs)
    y_pred_m = clf.predict(X)
    weights, alpha = get_updated_weights(y, y_pred_m, weights)
    alpha_list.append(alpha)
    y_pred_list.append(y_pred_m)

## Final Prediction and Evaluation

In [13]:
def get_final_prediction(alpha_list, y_pred_list):
    final_score = np.sign(np.sum([a * y for a, y in zip(alpha_list, y_pred_list)], axis=0))
    return final_score

y_final = get_final_prediction(alpha_list, y_pred_list)
f1_boosted = f1_score(y, y_final)
print(f"F1 Score after Boosting: {f1_boosted:.4f}")

F1 Score after Boosting: 0.9592
